## Read Segd
This code reads Segd files. It was originally created by Aditya Karan, UW ECE, in Jan. 2020 and revised by Shima Abadi, UW ECE, in March 2020. 

In [ ]:
# This version only supports reading files with format code 8058. Also this version only reads General Header #1 and General Header #2
# It does not read any information from any other general headers. It skips reading skew headers, external header blocks, and extended
# header

def readSegd(filename):
    import numpy as np
    import binascii
    import struct
    
    # Initializing variables
    class header:
        file_number = 0
        format_code = 0
        year = 0
        additional_header_blocks = 0
        julian_day = 0
        hour = 0
        minute = 0
        second = 0
        manufacturer_code = 0
        manufacturer_serial_number = 0
        bytes_per_scan = 0
        base_scan_interval = 0
        polarity_code = 0
        scans_in_block_exponent = 0
        scans_in_block_base = 0 
        record_type_code = 0
        record_length = 0
        scans_type_per_record = 0
        channel_set_per_scan_type = 0
        added_skew_fields = 0
        extended_header_length = 0
        external_header_length = 0
        SEGD_revision = 0
        general_trailers = 0
        channel_Set_channels = 0
        
    # This version only supports reading files with format code 8058. Also this version only reads General Header #1 and General Header #2
    # It does not read any information from any other general headers. It skips reading skew headers, external header blocks, and extended
    # header

    # Mention the location of the file
    with open(filename, 'rb') as f:
        content = f.read()
        temp = binascii.hexlify(content)

        ####### Part 1 - Reading Header Data 

        # Initialising and Computing Variables
        header.file_number = (temp[0:4]).decode('utf-8')
        header.format_code = int(temp[4:8])
        if header.format_code != 8058:
            raise Exception('This File can only read SegD files having format Code 8058')
        header.year = int('20' + temp[8:10].decode('utf-8'))
        header.additional_header_blocks = int(temp[22:23])
        header.julian_day = int(temp[23:26])
        header.hour = int(temp[26:28])
        header.minute = int(temp[28:30])
        header.second = int(temp[30:32])
        header.manufacturer_code = int(temp[32:34])
        header.manufacturer_serial_number = int(temp[34:38])
        header.bytes_per_scan = int(temp[38:44])
        header.base_scan_interval = int(temp[44:45])
        header.polarity_code = bin(int(temp[46:47].decode('utf-8')))
        header.scans_in_block_exponent = int(temp[48:49])
        header.scans_in_block_base = int(temp[49:50])
        header.record_type_code = bin(int(temp[50:51].decode('utf-8')))
        header.record_length = temp[51:54].decode('utf-8')
        if header.record_length != 'fff':
            header.record_length = (int(header.record_length) / 10) * 1.024

        header.scans_type_per_record = int(temp[54:56])
        if header.scans_type_per_record > 1:
            raise Exception('This version of readSegD only handles Seg-D files with a single scan type per record')

        header.channel_set_per_scan_type = temp[56:58].decode('utf-8')
        header.added_skew_fields = int(temp[58:60])
        header.extended_header_length = temp[60:62].decode('utf-8')
        header.external_header_length = temp[62:64].decode('utf-8') 

        # Reading General Header Block #2 
        if header.file_number == 'ff':
            header.file_number = int(temp[64:70].decode('utf-8'))

        if header.channel_set_per_scan_type == 'ff':
            header.channel_set_per_scan_type = temp[70:74].decode('utf-8', 16)

        if header.extended_header_length == 'ff':
            header.extended_header_length = int(temp[74:78].decode('utf-8'), 16)

        if header.external_header_length == 'ff':
            header.external_header_length = int(temp[78:82].decode('utf-8'), 16)

        header.SEGD_revision = float(temp[84:86]) 
        header.general_trailers = temp[86:90].decode('utf-8')
        if header.record_length == 'fff':
            header.record_length = int(temp[90:96])

        # This version does not read additional header blocks

        # Reading Scan Type Header
        count = 128 + header.additional_header_blocks * 32
        temp2 = temp[count:]
        num = temp2[0:2]
        header.channel_Set_channels = int(temp[260:264].decode('utf-8'))
        count = count + 64

        # Read Skew Fields
        if header.added_skew_fields > 0:
            count = count + header.added_skew_fields * 32 * 2
            print('Skipping Skew Fields')

        # Read Extended Header
        count = count + int(header.extended_header_length) * 32 * 2
        print('Skipping Extended Header Blocks')

        # Read External Length
        count = count + int(header.external_header_length) * 32 * 2
        print('Skipping External Header Blocks')

        ###### Part 2 - Reading Trace Data 

        tempcount = count

        ex = binascii.hexlify(content)

        tempcount = tempcount + 18
        search = int(ex[tempcount:tempcount + 2])
        tempcount = tempcount + 22

        tempcount = tempcount + 14
        samples_per_trace = (int(ex[tempcount: tempcount + 6], 16))
        tempcount = tempcount + 6 + 44

        data = np.zeros((samples_per_trace, header.channel_Set_channels))
        for i in range(0, header.channel_Set_channels):
            for j in range(0, samples_per_trace):
                tempstr = ex[tempcount:tempcount + 8].decode('utf-8')
                #print(tempstr)
                data[j][i] = struct.unpack('!f', bytes.fromhex(tempstr))[0]
                tempcount = tempcount + 8
            tempcount = tempcount + 40 + search * 32 * 2
    
    return header,data